In [ ]:
# Here we are using the same Skeleton of chatbot and improving it by adding loops and conditions to store the chat history.
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
import operator
from typing import List,Annotated
from langchain_core.messages import SystemMessage,HumanMessage,BaseMessage#(BaseMessage will add the flexibility of using HM , SM, AI Message in ChatBot )

from langgraph.checkpoint.memory import MemorySaver

In [ ]:
from langgraph.graph.message import add_messages
# add_messages is the built-in function to add messages to the state in langgraph which gives more flexibility for BaseMessage, instead of using operator.add 
class ChatbotState(TypedDict):
    messages:Annotated[List[BaseMessage],add_messages]   # this is reducer function to add all the message history into the list 

In [ ]:
model=ChatOpenAI()
def chat_node(state:ChatbotState):
    #take user query from the state, send it to LLM , response store to state
    messages=state['messages']
    response=model.invoke(messages)
    return {'messages':[response]}  # we have to pass the response as a list because in above messages is defined as List[BaseMessage]


In [ ]:
#add Checkpointer
Checkpointer=MemorySaver()

graph=StateGraph(ChatbotState)

# add nodes now
graph.add_node('chat_node',chat_node)
# Now add edges
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

workflow=graph.compile(checkpointer=Checkpointer)




In [ ]:
'''initial_state={
    'messages':[HumanMessage(content='What is the capital of India? ')]
}       
workflow.invoke(initial_state)['messages'][-1].content'''     # this is the normal invoke of the message and get the ai output

In [ ]:
'''while True:
    user_message=input("TypeHere: ")
    print("User:",user_message)
    if user_message.lower() in ['exit','quit','bye']:
        break
    response=workflow.invoke({'messages':[HumanMessage(content=user_message)]})
    print('AI: ',response['messages'][-1].content)                 '''                
# this is while loop to continously chat with AI This doesn't have any chat memory becauseof no thread and memory not included 
# so for that we have to include persistence which involves, checkpointer Memory saver , thread_id and also configurable



In [ ]:
'''thread_id = '1'
initial_state={
    'messages':[HumanMessage(content='What is the capital of India? ')]
}
config = {'configurable': {'thread_id': thread_id}}
workflow.invoke(initial_state, config=config)['messages'][-1].content'''   
# this is thread implementation example

In [ ]:
thread_id='1'
while True:
    user_message=input("TypeHere: ")
    print("User:",user_message)
    if user_message.lower() in ['exit','quit','bye']:
        print("Chatbot: Goodbye!")
        break
    config={'configurable':{'thread_id':thread_id}} 
    response=workflow.invoke({'messages':[HumanMessage(content=user_message)]},config=config)
    print("AI:",response['messages'][-1].content)                                             
# this involves threadid, while loop for continous chat , normal invoke with configurable 



In [ ]:
workflow.get_state(config=config) # to get all the caht history and all the messages with all the details
